## Generate Vts1 configurations from Alphafold predictions

In [1]:
import numpy as np
import os 
import sys
sys.path.insert(0, '../scripts')
from AF_to_LAMMPS import *
from gen_packmol import *
from gen_CG import *
from prody import *
from shape import *

(1) Get info about Vts1 from Alphafold. The Alphafold `.cif` structure file and predicted aligned error `.json` are required to assign order/disorder and type of disordered domain.

In [5]:
# make class
vts1 = AF2LAMMPS('vts1','../alphafold/AF-Q08831-F1-model_v4.cif','../alphafold/AF-Q08831-F1-predicted_aligned_error_v4.json')

# read coordinates and information about disorder
_,coords,sequence,_= vts1.read_coords()
plddt = vts1.get_plDDT()
pae = vts1.get_pae()
rig,flex = vts1.get_reg()
paired = vts1.get_paired_rig()
radii = np.ones(vts1.N) * 3.81/2
N = vts1.N

(2) Create randomized chain configurations

In [6]:
# generate id for different segments
combo = rig + flex
id_scramble = np.zeros(N)
count = 0
for i,c in enumerate(combo):
    id_scramble[np.arange(count,count+len(c))] = i
    count += len(c)
    
# create id_placed, which labels discrete regions 
id_placed = id_scramble[np.argsort(np.concatenate(combo))]
curr = 0
count = 0
for i in np.where(np.diff(id_placed) != 0)[0]:
    id_placed[curr:i+1] = count
    curr = i+1
    count += 1
id_placed[curr:-1] = count

# generate N chains

for citer in range(5):
    
    print(citer)
    # generate random config
    idx_placed, c_placed = generate_protein(coords,rig,flex,paired,radii,nr=1000)
    c_order = np.array(c_placed)[np.argsort(idx_placed)]
    
    # arbitrary assignment of atom and residue types
    at_name = ['CA'] * N
    res_name = [one_to_three[x] for x in sequence]
    temp_fact = radii
    
    # write pqr file
    with open('../pdb/c_%i.pqr'%citer,'w') as f:
        write_atom_pdb(c_order,'../pdb/c_%i.pqr'%citer,
                       at_name=at_name,
                       res_name=res_name,
                       temp_fact=temp_fact)

0
placing rigid regions
placing first IDR
placing IDR
1
placing rigid regions
placing first IDR
placing IDR
2
placing rigid regions
placing first IDR
placing IDR
3
placing rigid regions
placing first IDR
placing IDR
4
placing rigid regions
placing first IDR
placing IDR


(3a) Create an elastic network for Vts1 ordered domains: Use NMR data for Vts1-RBD

In [7]:
# get solution NMR data
fetchPDB('2ese')
p = parsePDB('2ese')
heavy = p.select('not hydrogen')
SAM = heavy.select('protein and name CA')[1:]

# Find root mean square fluctuation from mean position
avg_SAM = np.mean(SAM.getCoordsets(),axis=0)
_, rmsf_SAM = kabsch(SAM.getCoordsets())

# calculate B-factor
bf_SAM = 8 * np.pi **2 /3 * rmsf_SAM ** 2

@> Connecting wwPDB FTP server RCSB PDB (USA).
@> Downloading PDB files via FTP failed, trying HTTP.
@> 2ese downloaded (2ese.pdb.gz)
@> PDB download via HTTP completed (1 downloaded, 0 failed).
@> PDB file is found in working directory (2ese.pdb.gz).
@> 2100 atoms and 20 coordinate set(s) were parsed in 0.09s.


(3b) Use FitNMA to create a Delaunay triangulation mesh with relevant spring constants.

In [8]:
# create PDB of Vts1-RBD alpha carbons to be read in by FitNMA
with open('../pdb/Vts1_RBD_cA.pdb','w'):
    write_atom_pdb(avg_SAM,'../pdb/Vts1_RBD_cA.pdb',
                   temp_fact=bf_SAM)

# Run FitNMA 
fitnma_path = '/Applications/FitNMA/bin/FitNMA' # This may be different depending on your download
os.system('%s -i ../pdb/Vts1_RBD_cA.pdb -o Vts1RBD'%fitnma_path) 

i = 0 coord = 8.73806 5.06136 -4.46014
i = 1 coord = 11.5271 2.72436 -4.31014
i = 2 coord = 10.9101 -0.160638 -2.91314
i = 3 coord = 7.32306 -0.879638 -2.11914
i = 4 coord = 8.00606 -4.60664 -2.89814
i = 5 coord = 9.98606 -4.78964 0.410862
i = 6 coord = 8.88106 -7.38064 3.03986
i = 7 coord = 9.62406 -5.13864 6.03286
i = 8 coord = 7.43406 -2.35864 4.62286
i = 9 coord = 4.83106 -4.84464 3.33786
i = 10 coord = 4.62506 -6.42064 6.85286
i = 11 coord = 4.50006 -2.98164 8.55386
i = 12 coord = 1.95106 -0.812638 6.68986
i = 13 coord = 2.99606 2.48336 8.46686
i = 14 coord = 6.56806 2.11036 7.09586
i = 15 coord = 5.29806 1.14536 3.61286
i = 16 coord = 3.32706 4.44336 3.60786
i = 17 coord = 6.38806 6.33336 4.97286
i = 18 coord = 8.05606 5.70436 1.60086
i = 19 coord = 5.09506 7.52036 -0.122138
i = 20 coord = 4.77906 10.6014 2.18686
i = 21 coord = 1.22106 9.50136 2.86086
i = 22 coord = 2.07306 8.35236 6.41686
i = 23 coord = -0.723937 10.5714 7.79286
i = 24 coord = -3.19694 7.93036 6.77386
i = 25 coo

0

             176     60.460115     688.255931     60.460115     23290.857464
             177     55.307633     687.234640     55.307633     14204.123828
             178     50.244177     681.996916     50.244177     8626.422133
             179     47.987607     690.960158     47.987607     7217.382374
             180     46.554069     690.104145     46.554069     11483.024752
             181     44.125869     688.601351     44.125869     10929.807877
             182     43.333769     689.849329     43.333769     4821.461382
             183     43.002845     689.945195     43.002845     6171.166360
             184     42.717280     692.398835     42.717280     12125.175038
             185     41.895591     693.158389     41.895591     2395.080419
             186     41.760146     693.079778     41.760146     2155.912127
             187     41.616366     693.701995     41.616366     3386.012192
             188     41.530932     694.207597     41.530932     893.302932
        

             294     22.898996     697.061562     22.898996     323.963177
             295     22.894851     697.073421     22.894851     196.921183
             296     22.885527     697.103624     22.885527     677.802779
             297     22.870607     697.142257     22.870607     1252.216962
             298     22.836567     697.207238     22.836567     2050.316540
             299     22.763549     697.309025     22.763549     3012.032555
             300     22.604228     697.488251     22.604228     4158.967265
             301     22.283628     697.801827     22.283628     5379.233811
             302     21.667235     698.328741     21.667235     6645.780634
             303     20.532419     699.005221     20.532419     8375.347808
             304     18.627254     699.510851     18.627254     11512.646737
             305     17.220598     699.511509     17.220598     12597.913599
             306     15.706909     699.915689     15.706909     7635.693762
             

             412     5.748474     702.124805     5.748474     18.654963
             413     5.748460     702.128398     5.748460     16.693014
             414     5.748450     702.128415     5.748450     8.550110
             415     5.748446     702.127050     5.748446     4.485036
             416     5.748444     702.125286     5.748444     5.628310
             417     5.748443     702.125007     5.748443     5.188606
             418     5.748443     702.125021     5.748443     2.162768
             419     5.748443     702.125096     5.748443     1.522417
             420     5.748443     702.125300     5.748443     0.915626
             421     5.748443     702.125177     5.748443     1.571641
             422     5.748443     702.125250     5.748443     1.153206
             423     5.748443     702.125250     5.748443     1.815942
             424     5.748442     702.125151     5.748442     2.725720
             425     5.748442     702.124924     5.748442     3.587980
    

...

            1615     1.002980     730.295530     1.002980     10.396012
            1616     1.002977     730.296284     1.002977     7.838324
            1617     1.002974     730.295589     1.002974     9.320132
            1618     1.002971     730.294283     1.002971     11.270066
            1619     1.002968     730.293387     1.002968     8.954218
            1620     1.002965     730.293345     1.002965     5.382909
            1621     1.002963     730.294003     1.002963     5.055227
            1622     1.002961     730.294736     1.002961     9.545961
            1623     1.002958     730.297188     1.002958     13.331712
            1624     1.002954     730.299145     1.002954     18.627416
            1625     1.002946     730.301622     1.002946     20.442970
            1626     1.002930     730.306899     1.002930     13.654578
            1627     1.002926     730.307833     1.002926     54.605950
            1628     1.002914     730.306069     1.002914     19.84905

            1741     1.002791     730.325937     1.002791     2.927950
            1742     1.002791     730.326127     1.002791     1.429094
            1743     1.002791     730.326282     1.002791     2.341309
            1744     1.002790     730.326820     1.002790     3.869832
            1745     1.002790     730.328475     1.002790     6.780151
            1746     1.002789     730.328931     1.002789     3.222024
            1747     1.002789     730.329121     1.002789     2.093651
            1748     1.002789     730.329743     1.002789     0.880461
            1749     1.002789     730.329626     1.002789     1.344907
            1750     1.002789     730.329552     1.002789     2.362842
            1751     1.002789     730.329485     1.002789     3.007515
            1752     1.002789     730.329284     1.002789     3.602753
            1753     1.002789     730.329586     1.002789     1.806159
            1754     1.002789     730.329533     1.002789     1.050615
      

            1867     1.002709     730.348005     1.002709     7.842498
            1868     1.002708     730.347552     1.002708     5.186647
            1869     1.002708     730.347712     1.002708     1.976325
            1870     1.002707     730.348097     1.002707     2.514547
            1871     1.002707     730.348319     1.002707     5.284569
            1872     1.002707     730.348274     1.002707     2.340369
            1873     1.002707     730.348189     1.002707     2.111230
            1874     1.002707     730.347917     1.002707     4.130669
            1875     1.002706     730.347768     1.002706     4.710236
            1876     1.002706     730.348171     1.002706     4.863753
            1877     1.002706     730.347692     1.002706     3.064561
            1878     1.002706     730.347936     1.002706     1.737065
            1879     1.002705     730.348274     1.002705     4.878630
            1880     1.002705     730.348392     1.002705     6.911716
      

            1987     1.002683     730.343794     1.002683     1.039254
            1988     1.002683     730.343594     1.002683     1.988886
            1989     1.002683     730.343533     1.002683     3.190563
            1990     1.002683     730.343364     1.002683     2.051205
            1991     1.002683     730.343124     1.002683     1.540013
            1992     1.002683     730.342911     1.002683     1.763544
            1993     1.002683     730.342702     1.002683     2.379884
            1994     1.002683     730.342696     1.002683     2.346488
            1995     1.002683     730.342971     1.002683     4.059540
            1996     1.002682     730.343325     1.002682     3.018895
            1997     1.002682     730.343904     1.002682     1.889299
            1998     1.002682     730.344833     1.002682     1.840816
            1999     1.002682     730.345441     1.002682     2.752008
            2000     1.002682     730.345672     1.002682     3.918276
 
Mean

0

(3c) Load data from elastic network model

In [10]:
# load data
r_enm = np.loadtxt('Vts1RBD.enm',dtype=object)

# extract bond topology and spring constant
pi = [int(x) for x in r_enm[:,0]]
pj = [int(x) for x in r_enm[:,-2]]
k = [float(x) for x in r_enm[:,-1]]
r_enm_data= np.array((pi,pj,k),dtype=object)


# create array to be written to lammps bond file
r_kconst = np.zeros((r_enm_data.shape[1],4),dtype=object)
r_kconst[:,:3] = r_enm_data.T
r_kconst[:,3] = np.sqrt(np.sum((avg_SAM[(r_kconst[:,0]-1).tolist()] - avg_SAM[(r_kconst[:,1]-1).tolist()]) ** 2,axis=1))

# remove non-sensical bonds to self and list bond types as they will appear in LAMMPS bond file
r_kconst = np.delete(r_kconst,np.where(r_kconst[:,0] == r_kconst[:,1])[0].tolist(),0)

# change indices to match full length protein
r_kconst[:,0] = r_kconst[:,0] + 443
r_kconst[:,1] = r_kconst[:,1] + 443

# get bond types
r_bond_types = np.concatenate((np.arange(1,len(r_kconst)+1).reshape(len(r_kconst),1),r_kconst[:,-2:]),axis=1).astype(object)

(4a) Create an elastic network for Vts1 ordered domains: Since NMR data is not available for the alpha-helical region in the Vts1-IDR, we will approximate using values based on plDDT

In [12]:
def bf_from_plddt(plddt):
    '''
    Estimates B factor from heuristic from Hiranuma, N., Park, H., Baek, M. et al. (2021).
    
    '''
    rmsd = 1.5 * np.exp(4 * ((0.7 - plddt/100)))
    return 8 * np.pi **2 /3 * rmsd ** 2 

(4b) Use FitNMA again for this structural region

In [13]:
# create PDB of Vts1-RBD alpha carbons to be read in by FitNMA
helix_c = coords[rig[0]]
bf_helix = bf_from_plddt(plddt[-1][rig[0]])
with open('../pdb/Vts1_helix_cA.pdb','w'):
    write_atom_pdb(helix_c,'../pdb/Vts1_helix_cA.pdb',
                   temp_fact=bf_helix)

# Run FitNMA 
fitnma_path = '/Applications/FitNMA/bin/FitNMA'
os.system('%s -i ../pdb/Vts1_helix_cA.pdb -o Vts1_helix'%fitnma_path) 

i = 0 coord = 14.4147 2.13817 2.95694
i = 1 coord = 12.5447 2.99917 6.22994
i = 2 coord = 12.8667 6.75017 5.40694
i = 3 coord = 11.5757 6.06117 1.83394
i = 4 coord = 8.54368 4.08117 3.18794
i = 5 coord = 7.71468 6.95817 5.60294
i = 6 coord = 8.02668 9.50317 2.71894
i = 7 coord = 5.82468 7.21617 0.509936
i = 8 coord = 3.15268 6.95117 3.28794
i = 9 coord = 3.32268 10.7722 3.81494
i = 10 coord = 3.00768 11.4012 0.0309362
i = 11 coord = 0.0676809 8.91117 -0.346064
i = 12 coord = -1.67332 10.6732 2.61394
i = 13 coord = -1.43532 13.9972 0.631936
i = 14 coord = -3.18232 12.5052 -2.49006
i = 15 coord = -6.96732 12.6382 -3.17406
i = 16 coord = -8.81632 9.28017 -2.52706
i = 17 coord = -9.05032 8.53217 -6.31806
i = 18 coord = -5.26132 9.13317 -6.64606
i = 19 coord = -4.64232 7.03817 -3.45806
i = 20 coord = -6.63832 4.12717 -5.00106
i = 21 coord = -4.73632 4.56217 -8.33406
i = 22 coord = -1.34032 4.48617 -6.48206
i = 23 coord = -2.43132 1.47617 -4.34906
i = 24 coord = -3.57632 -0.45583 -7.50206
i 

0

             242     44.267368     195.759568     44.267368     586.043460
             243     44.247090     195.759159     44.247090     678.189224
             244     44.227985     195.765202     44.227985     543.197792
             245     44.205075     195.763290     44.205075     762.271853
             246     44.193402     195.740955     44.193402     1202.968194
             247     44.170715     195.732141     44.170715     769.802661
             248     44.143415     195.709202     44.143415     834.606903
             249     44.113017     195.692973     44.113017     857.243934
             250     44.045674     195.676359     44.045674     873.519077
             251     43.980510     195.721672     43.980510     2335.224177
             252     43.897193     195.709298     43.897193     1358.399023
             253     43.843367     195.741571     43.843367     1146.738237
             254     43.809440     195.775651     43.809440     887.095735
             255     

             385     32.596626     196.096473     32.596626     384.375458
             386     32.589238     196.093283     32.589238     448.002453
             387     32.569658     196.083389     32.569658     651.963467
             388     32.557026     196.079172     32.557026     848.034087
             389     32.544034     196.074550     32.544034     453.209699
             390     32.529688     196.071696     32.529688     497.880401
             391     32.494125     196.057216     32.494125     753.278427
             392     32.458971     196.038497     32.458971     1385.596218
             393     32.405998     196.011608     32.405998     1228.836772
             394     32.390144     196.001105     32.390144     1888.724107
             395     32.318884     195.957860     32.318884     578.650681
             396     32.302616     195.959065     32.302616     406.491820
             397     32.297087     195.952958     32.297087     620.120283
             398     3

             530     23.461207     196.630238     23.461207     384.353346
             531     23.459066     196.632583     23.459066     349.446537
             532     23.457249     196.630175     23.457249     313.791093
             533     23.448974     196.628178     23.448974     303.260944
             534     23.436418     196.625601     23.436418     478.662703
             535     23.417889     196.635285     23.417889     813.958643
             536     23.397943     196.655962     23.397943     708.048824
             537     23.361846     196.699862     23.361846     477.686376
             538     23.332509     196.736912     23.332509     399.726727
             539     23.310634     196.786126     23.310634     1034.442781
             540     23.265574     196.806790     23.265574     813.598125
             541     23.198268     196.808913     23.198268     792.875559
             542     23.174768     196.813027     23.174768     2032.898543
             543     23

...

            1461     9.191475     199.742267     9.191475     338.444352
            1462     9.179709     199.729316     9.179709     748.658207
            1463     9.164814     199.713033     9.164814     1026.296517
            1464     9.148565     199.689180     9.148565     1095.323837
            1465     9.132958     199.674561     9.132958     911.281598
            1466     9.121831     199.679229     9.121831     266.273135
            1467     9.115563     199.690434     9.115563     310.869696
            1468     9.112547     199.692426     9.112547     341.613896
            1469     9.109985     199.701665     9.109985     182.597425
            1470     9.109079     199.700623     9.109079     181.405313
            1471     9.108481     199.699483     9.108481     86.938683
            1472     9.107828     199.702041     9.107828     217.504392
            1473     9.106794     199.702216     9.106794     93.603766
            1474     9.104732     199.702422     9.

            1609     7.988594     200.471452     7.988594     145.239944
            1610     7.986435     200.477431     7.986435     161.559234
            1611     7.984736     200.484895     7.984736     142.742566
            1612     7.983657     200.505117     7.983657     293.250444
            1613     7.981188     200.513618     7.981188     196.636239
            1614     7.976190     200.532483     7.976190     234.607937
            1615     7.966945     200.568515     7.966945     464.678782
            1616     7.951469     200.626648     7.951469     661.462960
            1617     7.931459     200.697021     7.931459     847.512550
            1618     7.922917     200.711455     7.922917     441.720207
            1619     7.907266     200.740064     7.907266     460.903401
            1620     7.895006     200.731894     7.895006     652.428548
            1621     7.882294     200.709305     7.882294     854.485046
            1622     7.863530     200.673261     7.

            1756     6.571730     201.594858     6.571730     796.426957
            1757     6.567365     201.614268     6.567365     716.857150
            1758     6.558102     201.643928     6.558102     363.213588
            1759     6.552639     201.655383     6.552639     314.282654
            1760     6.546651     201.662121     6.546651     227.591692
            1761     6.540490     201.650837     6.540490     167.248516
            1762     6.538042     201.636411     6.538042     428.435031
            1763     6.533914     201.616032     6.533914     304.546263
            1764     6.531553     201.604156     6.531553     283.561319
            1765     6.528847     201.593855     6.528847     302.125356
            1766     6.526920     201.585433     6.526920     295.696430
            1767     6.523922     201.587422     6.523922     193.647274
            1768     6.519059     201.597404     6.519059     373.879890
            1769     6.511719     201.612778     6.

            1901     6.253724     201.984143     6.253724     290.520735
            1902     6.245384     201.991003     6.245384     331.669734
            1903     6.238812     201.997785     6.238812     507.785955
            1904     6.220712     202.023816     6.220712     1029.452015
            1905     6.212362     202.051991     6.212362     1066.993342
            1906     6.197502     202.065043     6.197502     645.454593
            1907     6.188877     202.075442     6.188877     226.236433
            1908     6.187707     202.074672     6.187707     67.560350
            1909     6.186821     202.072410     6.186821     120.045762
            1910     6.185624     202.068363     6.185624     230.469656
            1911     6.184409     202.062949     6.184409     197.501678
            1912     6.183982     202.058426     6.183982     266.952760
            1913     6.183073     202.051897     6.183073     107.632618
            1914     6.182420     202.045211     6

0

(4c) Extract data from FitNMA for Vts1 helix

In [14]:
# load data
h_enm = np.loadtxt('Vts1_helix.enm',dtype=object)

# extract bond topology and spring constant
pi = [int(x) for x in h_enm[:,0]]
pj = [int(x) for x in h_enm[:,-2]]
k = [float(x) for x in h_enm[:,-1]]
h_enm_data= np.array((pi,pj,k),dtype=object)


# create array to be written to lammps bond file
h_kconst = np.zeros((h_enm_data.shape[1],4),dtype=object)
h_kconst[:,:3] = h_enm_data.T
h_kconst[:,3] = np.sqrt(np.sum((avg_SAM[(h_kconst[:,0]-1).tolist()] - avg_SAM[(h_kconst[:,1]-1).tolist()]) ** 2,axis=1))

# remove non-sensical bonds to self and list bond types as they will appear in LAMMPS bond file
h_kconst = np.delete(h_kconst,np.where(h_kconst[:,0] == h_kconst[:,1])[0].tolist(),0)

# change indices to match full length protein
h_kconst[:,0] = h_kconst[:,0] + 187
h_kconst[:,1] = h_kconst[:,1] + 187

h_bond_types = np.concatenate((np.arange(1,len(h_kconst)+1).reshape(len(h_kconst),1),h_kconst[:,-2:]),axis=1).astype(object)

(4d) Combine bond information and add connections between IDR residues

In [15]:
idr1 = np.arange(1,188)
idr2 = np.arange(234,444)
idrc = np.concatenate([idr1,idr2])
idr_k = 9.60
idr_eq = 3.81
i_kconst = np.zeros((len(idrc),4),dtype=object)
i_kconst[:,0] = idrc
i_kconst[:,1] = idrc+1
i_kconst[:,2] = idr_k
i_kconst[:,3] = idr_eq
i_bond_types = np.array([[1,idr_k,idr_eq]]) 

In [16]:
kconst = np.concatenate([i_kconst,h_kconst,r_kconst])
bond_types = np.concatenate([i_bond_types,h_bond_types,r_bond_types])
bond_types[:,0] = np.arange(1,len(bond_types)+1)

(5a) Pack initial configuration into LAMMPS box

In [17]:
# Generate Packmol input
bl = 300
nchains = 2
out = '../pdb/2_vts1.pdb'
with open('2_vts1.in','w') as f:
    
    # write header
    f.write('filetype pdb\noutput %s\ntolerance 1.0\n'%(out))
    
    # iterate through chain conformations and add to box
    for i in range(2):
        f.write('\nstructure ../pdb/c_%i.pqr\n\tnumber %i\n\tinside box 0. 0. 0. %f % f %f\nend structure'%(i,1,bl,bl,bl))

(5b) Run packmol

In [18]:
packmol_path = '/Applications/packmol/packmol'
os.system('%s < 2_vts1.in'%packmol_path) 


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 20.4.0 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Seed for random number generator:      1234567
  Output file: ../pdb/2_vts1.pdb
  Reading coordinate file: ../pdb/c_0.pqr
  Reading coordinate file: ../pdb/c_1.pqr
  Number of independent structures:            2
  The structures are: 
  Structure            1 :../pdb/c_0.pqr(         523  atoms)
  Structure            2 :../pdb/c_1.pqr(         523  atoms)
  Maximum number of GENCAN loops for all molecule packing:          400
  Total number of restrictions:            2
 

0

(6a) Use LAMMPS to squeeze box to desired volume fraction. Note that typically Mpipi has 40 atom types, half of which correspond to regions within ordered domains. We will only use the first 20 types in our simulations, as we will treat inter-molecular interactions the same irrespective of order/disorder and impose intramolecular structure of ordered regions with an elastic network model.

In [20]:
# Define name, box size, atom types
datafile = '../data/data.2_vts1_pre_squeeze'
n_atom_types = 40
aad = np.loadtxt('../params/aa_index.txt',dtype=object)
aad = dict(zip(aad[:,0],[int(x) for x in aad[:,1]]))
seq_id = np.array([aad[x] for x in res_name])
seq_len = N
xbound, ybound, zbound = (bl,bl,bl)
masses = np.loadtxt('../params/masses.txt').astype(object)

# define bond information
n_bond_types = len(r_bond_types) + len(h_bond_types)

# grab coordinates from Packmol output
pdb_lines = open('../pdb/2_vts1.pdb').readlines()[5:-1]
coords = np.zeros((seq_len * nchains,3))
for i,l in enumerate(pdb_lines):
    coords[i] = [float(l[31:38]),
                 float(l[39:46]),
                 float(l[47:54])]
    
# generate matrix to write to ATOM section of LAMMPS data file
atoms = np.zeros((len(coords),7),dtype=object)
atoms[:,0] = 1 + np.arange(len(coords))
atoms[:,1] = np.hstack([np.ones(seq_len).astype(int) * (i+1) for i in range(nchains) ])
atoms[:,2] = np.tile(seq_id,nchains)
atoms[:,-3:] = coords

# generate matrix to write to BOND section of LAMMPS data file
bonds_mon = np.zeros((len(kconst),4))
bonds_mon[:,-2] = kconst[:,0]
bonds_mon[:,-1] = kconst[:,1]
s1,s2,s3 = np.split(bonds_mon,np.cumsum([len(i_kconst),len(h_kconst),len(r_kconst)]))[:-1]
s1[:,1] = 1
s2[:,1] = np.arange(2,2+len(s2))
s3[:,1] = np.arange(len(s2)+2,len(s2)+2 + len(s3))
bonds_mon = np.concatenate([s1,s2,s3]) 
bonds = np.concatenate([bonds_mon]*nchains)
bid = np.concatenate([bonds_mon[:,-2:] +  523 * i for i in range(nchains)])
bonds[:,-2:] = bid
bonds[:,0] = np.arange(1,len(bonds)+1)
bonds = bonds.astype(int)
# write data files

with open(datafile,'w'):
    write_lammps_data(datafile,
                      atoms,
                      n_atom_types,
                      [0,xbound],
                      [0,ybound],
                      [0,zbound],
                      list(masses[:,1]) * 2,
                      n_bond_types=len(bond_types),
                      bonds=bonds)

(6b) Write bond potential file

In [21]:
count = 1
with open('../input/Vts1.bond','w') as f:
    f.write('bond_style harmonic\n')
    f.write('bond_coeff %i %f %f\n'%(1,9.6,3.81))
    for i,line in enumerate(kconst):
        if (line[2] == 9.6) & (line[3] == 3.81):
            continue
        f.write('bond_coeff %i %f %f\n'%(count+1,line[-2],line[-1]))
        count+=1

(6c) Write a parameter file for LAMMPS squeezing simulations. The "parambase.squeeze" file is a template which has the LAMMPS codes that deforms the box over a set amount of time to generate dense phase simulations. Note that these parameters likely must be changed based on user preferences and protein size.

In [24]:
# configure parameters
data_file_in = 'data/data.2_vts1_pre_squeeze'
seed_in = 123
pair_interactions_in = 'input/set_pair_param-Mpipi_struct.dat'
bond_interactions_in = 'input/Vts1.bond'
final_length_in = 220
vfrac_init_in = (np.pi * 4 /3 * (3.5/2) ** 3 * N * 100) / bl ** 3
vfrac_fin_in = (np.pi * 4 /3 * (3.5/2) ** 3 * N * 100) / final_length_in ** 3
equib_time_in = 25000
run_name = 'data.vts1_squeeze'
temp_lab_in = 303
thermo_increment_in = 1
dump_increment_in = 500

# write parambase file
param_lines = open('../input/parambase.squeeze').readlines()
pl = copy(param_lines)
pl[0] = pl[0].replace('data_file_in', data_file_in)
pl[1] = pl[1].replace('seed_in',str(seed_in))
pl[2] = pl[2].replace('pair_interactions_in',pair_interactions_in)
pl[3] = pl[3].replace('bond_interactions_in',bond_interactions_in)
pl[4] = pl[4].replace('vfrac_init_in',str(vfrac_init_in))
pl[5] = pl[5].replace('vfrac_fin_in',str(vfrac_fin_in))
pl[6] = pl[6].replace('equib_time_in',str(equib_time_in))
pl[7] = pl[7].replace('run_name_in',run_name)
pl[8] = pl[8].replace('final_length_in',str(final_length_in))
pl[12] = pl[12].replace('temp_lab_in',str(temp_lab_in))
pl[14] = pl[14].replace('thermo_increment_in',str(thermo_increment_in))
pl[15] = pl[15].replace('dump_increment_in',str(dump_increment_in))
with open('../input/parambase.' + run_name,'w') as f:
    for line in pl:
        f.write(line)

Run squeeze via:
`lmp_mpi -v param input/parambase.data.vts1_squeeze -in input/in.squeeze `